In [2]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import sys
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import loompy
import velocyto as vcy
import logging
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.interpolate import interp1d
import cytograph as cg
import loompy
import numpy as np
import scipy.sparse as sparse
import torch
from torch.distributions.poisson import Poisson
from torch.autograd import Variable
from torch.nn import MSELoss
from tqdm import trange
from types import SimpleNamespace
from matplotlib.colors import Normalize
from matplotlib.collections import LineCollection

2019-03-25 16:22:02,579 DEBUG: backend module://ipykernel.pylab.backend_inline version unknown


In [23]:
with loompy.connect("/Users/stelin/20190322/CephalicFiltered.loom") as ds:
    print(ds[ds.ra.Gene == "mt-Nd6", :].sum())

0.0


In [24]:
[g for g in genes if g.startswith("mt")]

['mt-Nd6',
 'mt-Atp8',
 'mt-Cytb',
 'mt-Co3',
 'mt-Nd3',
 'mt-Atp6',
 'mt-Co2',
 'mt-Nd4',
 'mt-Nd1',
 'mt-Nd5',
 'mt-Co1',
 'mt-Nd2',
 'mt-Nd4l']

In [26]:
int("E9"[1:]) < int("E12"[1:])

True

In [29]:
cortex = np.loadtxt("/Users/stelin/Downloads/sourcedata/cortex_svz_counts.csv", delimiter=",", skiprows=1)

In [30]:
with open("/Users/stelin/Downloads/sourcedata/cortex_svz_counts.csv") as f:
    genes = f.readline().split(",")

In [32]:
len(genes)

10000

In [33]:
cortex.shape

(913, 10000)

In [36]:
loompy.create("/Users/stelin/loom/seqplus_cortex_svz.loom", cortex.T, {"Accession": np.array(genes), "Gene": np.array(genes)}, {"CellID": np.arange(cortex.shape[0])})

In [39]:
with loompy.connect("/Users/stelin/loom/seqplus_cortex_svz.loom") as ds:
    print(ds.layers.keys())

['']


In [40]:
import cytograph as cg

In [41]:


cc_genes_human = np.array([
	'ABHD3', 'AC016205.1', 'AC073529.1', 'AC084033.3', 'AC087632.1',
	'AC091057.6', 'AC097534.2', 'AC099850.2', 'AC135586.2', 'ACAA2',
	'ACADM', 'ACP1', 'ACTL6A', 'ACYP1', 'ADCY3', 'ADD3', 'ADK', 'AHCY',
	'AKIRIN2', 'AKR7A2', 'AL359513.1', 'AL449266.1', 'AL513165.2',
	'ANAPC11', 'ANLN', 'ANP32A', 'ANP32B', 'ANP32E', 'AP001347.1',
	'APOLD1', 'ARHGAP11A', 'ARHGEF39', 'ARID1A', 'ARL6IP1', 'ARL6IP6',
	'ARMC1', 'ARPP19', 'ASF1B', 'ASPM', 'ASRGL1', 'ATAD2', 'ATAD5',
	'ATP1B3', 'AURKA', 'AURKB', 'BANF1', 'BARD1', 'BAZ1A', 'BAZ1B',
	'BIRC5', 'BLM', 'BORA', 'BRCA1', 'BRCA2', 'BRD8', 'BRIP1', 'BTG3',
	'BUB1', 'BUB1B', 'BUB3', 'C11orf58', 'C19orf48', 'C1orf112',
	'C1orf35', 'C21orf58', 'C5orf34', 'CACYBP', 'CALM2', 'CAMTA1',
	'CARHSP1', 'CBX1', 'CBX3', 'CBX5', 'CCAR1', 'CCDC14', 'CCDC167',
	'CCDC18', 'CCDC34', 'CCDC77', 'CCNA1', 'CCNA2', 'CCNB1', 'CCNB2',
	'CCNE2', 'CCNF', 'CCT4', 'CCT5', 'CDC20', 'CDC25B', 'CDC25C',
	'CDC27', 'CDC45', 'CDC6', 'CDC7', 'CDCA2', 'CDCA3', 'CDCA4',
	'CDCA7L', 'CDCA8', 'CDK1', 'CDK19', 'CDK2', 'CDK4', 'CDK5RAP2',
	'CDKAL1', 'CDKN1B', 'CDKN2C', 'CDKN3', 'CDT1', 'CENPA', 'CENPC',
	'CENPE', 'CENPF', 'CENPH', 'CENPI', 'CENPJ', 'CENPK', 'CENPL',
	'CENPM', 'CENPN', 'CENPO', 'CENPP', 'CENPQ', 'CENPU', 'CENPW',
	'CENPX', 'CEP112', 'CEP128', 'CEP135', 'CEP192', 'CEP295', 'CEP55',
	'CEP57', 'CEP57L1', 'CEP70', 'CETN3', 'CFAP20', 'CFL2', 'CGGBP1',
	'CHAF1A', 'CHCHD2', 'CHEK1', 'CHEK2', 'CHRAC1', 'CIP2A', 'CIT',
	'CKAP2', 'CKAP2L', 'CKAP5', 'CKLF', 'CKS1B', 'CKS2', 'CLSPN',
	'CMC2', 'CMSS1', 'CNIH4', 'CNN3', 'CNTLN', 'CNTRL', 'COA1',
	'COMMD4', 'COX8A', 'CSE1L', 'CTCF', 'CTDSPL2', 'CWF19L2', 'CYB5B',
	'CYCS', 'DACH1', 'DBF4', 'DBF4B', 'DBI', 'DCAF7', 'DCP2', 'DCXR',
	'DDAH2', 'DDX39A', 'DDX46', 'DEK', 'DEPDC1', 'DEPDC1B', 'DESI2',
	'DHFR', 'DIAPH3', 'DKC1', 'DLEU2', 'DLGAP5', 'DNA2', 'DNAJB1',
	'DNAJC9', 'DNMT1', 'DPM1', 'DR1', 'DSCC1', 'DSN1', 'DTL', 'DTYMK',
	'DUSP16', 'DUT', 'DYNLL1', 'DYRK1A', 'E2F3', 'E2F7', 'E2F8',
	'ECT2', 'EED', 'EEF1D', 'EID1', 'EIF1AX', 'EIF2S2', 'EIF4A3',
	'EIF4E', 'EIF5', 'EMC9', 'ENAH', 'ENO1', 'ENY2', 'ERH', 'ESCO2',
	'EWSR1', 'EXOSC8', 'EZH2', 'FAM111B', 'FAM122B', 'FAM72C',
	'FAM72D', 'FAM83D', 'FANCB', 'FANCD2', 'FANCI', 'FANCL', 'FBL',
	'FBXL5', 'FBXO5', 'FDPS', 'FDX1', 'FEN1', 'FGFR1OP', 'FILIP1L',
	'FOXM1', 'FUS', 'FUZ', 'FXR1', 'FZR1', 'G2E3', 'G3BP1', 'GABPB1',
	'GAPDH', 'GAS2L3', 'GEMIN2', 'GEN1', 'GGCT', 'GGH', 'GINS2',
	'GLO1', 'GMNN', 'GMPS', 'GNG5', 'GPBP1', 'GPSM2', 'GTSE1', 'H1FX',
	'H2AFV', 'H2AFX', 'H2AFY', 'H2AFZ', 'HACD3', 'HADH', 'HAT1',
	'HAUS1', 'HAUS6', 'HAUS8', 'HDAC2', 'HDGF', 'HELLS', 'HES1',
	'HINT1', 'HIRIP3', 'HIST1H1A', 'HIST1H1C', 'HIST1H1D', 'HIST1H2BH',
	'HIST1H4C', 'HIST2H2AC', 'HJURP', 'HMG20B', 'HMGA1', 'HMGA2',
	'HMGB1', 'HMGB2', 'HMGB3', 'HMGN1', 'HMGN2', 'HMGN3', 'HMGN5',
	'HMGXB4', 'HMMR', 'HNRNPA0', 'HNRNPA1', 'HNRNPA2B1', 'HNRNPA3',
	'HNRNPAB', 'HNRNPC', 'HNRNPD', 'HNRNPDL', 'HNRNPF', 'HNRNPH3',
	'HNRNPK', 'HNRNPLL', 'HNRNPM', 'HNRNPU', 'HNRNPUL1', 'HP1BP3',
	'HPF1', 'HSD17B11', 'HSP90AA1', 'HSP90B1', 'HSPA13', 'HSPA1B',
	'HSPB11', 'HSPD1', 'HSPE1', 'HYLS1', 'IDH2', 'IFT122', 'IGF2BP3',
	'IKBIP', 'ILF2', 'ILF3', 'ILVBL', 'IMMP1L', 'INCENP', 'IPO5',
	'IQGAP3', 'ISCA2', 'ISOC1', 'ITGAE', 'ITGB3BP', 'JADE1', 'JPT1',
	'KATNBL1', 'KCTD9', 'KIAA0586', 'KIF11', 'KIF14', 'KIF15',
	'KIF18A', 'KIF18B', 'KIF20A', 'KIF20B', 'KIF22', 'KIF23', 'KIF2C',
	'KIF4A', 'KIF5B', 'KIFC1', 'KMT5A', 'KNL1', 'KNSTRN', 'KPNA2',
	'KPNB1', 'LARP7', 'LBR', 'LCORL', 'LDHA', 'LDHB', 'LIG1', 'LIN52',
	'LINC01224', 'LINC01572', 'LMNB1', 'LMNB2', 'LRR1', 'LSM14A',
	'LSM2', 'LSM3', 'LSM4', 'LSM5', 'LSM6', 'LSM7', 'LSM8', 'LUC7L2',
	'MAD2L1', 'MAGI1', 'MAGOH', 'MAGOHB', 'MAPK1IP1L', 'MAPRE1',
	'MARCKS', 'MASTL', 'MBNL2', 'MCM10', 'MCM2', 'MCM3', 'MCM4',
	'MCM5', 'MCM7', 'MDH1', 'MED30', 'MELK', 'MGME1', 'MIS18A',
	'MIS18BP1', 'MKI67', 'MMS22L', 'MND1', 'MNS1', 'MORF4L2',
	'MPHOSPH9', 'MRE11', 'MRPL18', 'MRPL23', 'MRPL47', 'MRPL51',
	'MRPL57', 'MRPS34', 'MTFR2', 'MYBL2', 'MYEF2', 'MZT1', 'MZT2B',
	'NAA38', 'NAA50', 'NAE1', 'NAP1L1', 'NAP1L4', 'NASP', 'NCAPD2',
	'NCAPD3', 'NCAPG', 'NCAPG2', 'NCAPH', 'NCL', 'NDC1', 'NDC80',
	'NDE1', 'NDUFA6', 'NDUFAF3', 'NDUFS6', 'NEDD1', 'NEIL3', 'NEK2',
	'NELFE', 'NENF', 'NFATC3', 'NFYB', 'NIPBL', 'NMU', 'NONO', 'NOP56',
	'NOP58', 'NRDC', 'NSD2', 'NSMCE2', 'NSMCE4A', 'NUCKS1', 'NUDC',
	'NUDCD2', 'NUDT1', 'NUDT15', 'NUDT21', 'NUDT5', 'NUF2', 'NUP107',
	'NUP35', 'NUP37', 'NUP50', 'NUP54', 'NUSAP1', 'ODC1', 'ODF2',
	'OIP5', 'ORC6', 'PA2G4', 'PAICS', 'PAIP2', 'PAK4', 'PAPOLA',
	'PARP1', 'PARPBP', 'PAXX', 'PBK', 'PCBD2', 'PCBP2', 'PCM1', 'PCNA',
	'PCNP', 'PDS5B', 'PHF19', 'PHF5A', 'PHGDH', 'PHIP', 'PIF1',
	'PIMREG', 'PIN1', 'PKM', 'PLCB1', 'PLGRKT', 'PLIN3', 'PLK1',
	'PLK4', 'PMAIP1', 'PNISR', 'PNN', 'PNRC2', 'POC1A', 'POLD2',
	'POLD3', 'POLE2', 'POLQ', 'POLR2C', 'POLR2D', 'POLR2G', 'POLR2J',
	'POLR2K', 'POLR3K', 'PPIA', 'PPIG', 'PPIH', 'PPP1CC', 'PPP2R3C',
	'PPP2R5C', 'PPP6R3', 'PRC1', 'PRDX3', 'PRIM1', 'PRIM2', 'PRKDC',
	'PRPF38B', 'PRPSAP1', 'PRR11', 'PSIP1', 'PSMA3', 'PSMA4', 'PSMB2',
	'PSMB3', 'PSMC3', 'PSMC3IP', 'PSMD10', 'PSMD14', 'PSMG2', 'PSRC1',
	'PTBP1', 'PTGES3', 'PTMA', 'PTMS', 'PTTG1', 'PUF60', 'RAB8A',
	'RACGAP1', 'RAD21', 'RAD51AP1', 'RAD51B', 'RAD51C', 'RAN',
	'RANBP1', 'RANGAP1', 'RASSF1', 'RBBP4', 'RBBP8', 'RBL1', 'RBM17',
	'RBM39', 'RBM8A', 'RBMX', 'RCC1', 'RDX', 'REEP4', 'RFC1', 'RFC2',
	'RFC3', 'RFC4', 'RFWD3', 'RHEB', 'RMI2', 'RNASEH2B', 'RNASEH2C',
	'RNF138', 'RNF168', 'RNF26', 'RNPS1', 'RPA1', 'RPA3', 'RPL35',
	'RPL39L', 'RPLP0', 'RPLP1', 'RPLP2', 'RPN2', 'RPP30', 'RPS15',
	'RPS16', 'RPS20', 'RPS21', 'RPSA', 'RRM1', 'RSRC1', 'RSRC2',
	'RTKN2', 'RUVBL2', 'SAC3D1', 'SAE1', 'SAP18', 'SAPCD2', 'SCAF11',
	'SCLT1', 'SDHAF3', 'SELENOK', 'SEM1', 'SEPHS1', 'SEPT10', 'SEPT2',
	'SEPT7', 'SERBP1', 'SET', 'SF1', 'SF3B2', 'SFPQ', 'SGO1', 'SGO2',
	'SHCBP1', 'SINHCAF', 'SIVA1', 'SKA1', 'SKA2', 'SKA3', 'SLBP',
	'SLC20A1', 'SLC25A3', 'SLTM', 'SMC1A', 'SMC2', 'SMC3', 'SMC4',
	'SMC5', 'SMCHD1', 'SNAPC1', 'SNRNP25', 'SNRNP40', 'SNRNP70',
	'SNRPA', 'SNRPA1', 'SNRPB', 'SNRPC', 'SNRPD1', 'SNRPD2', 'SNRPD3',
	'SNRPE', 'SNRPF', 'SNRPG', 'SON', 'SPAG5', 'SPATA5', 'SPC25',
	'SPCS2', 'SPDL1', 'SREK1', 'SRI', 'SRP9', 'SRRM1', 'SRSF1',
	'SRSF10', 'SRSF11', 'SRSF2', 'SRSF3', 'SRSF4', 'SRSF7', 'SSB',
	'SSBP1', 'SSNA1', 'SSRP1', 'ST13', 'STAG1', 'STIL', 'STIP1',
	'STK17B', 'STK3', 'STOML2', 'SUGP2', 'SUMO1', 'SUMO3', 'SUPT16H',
	'SUV39H2', 'SUZ12', 'SYNE2', 'TACC3', 'TBC1D31', 'TBC1D5', 'TDP1',
	'TEAD1', 'TEX30', 'TFDP1', 'THRAP3', 'TICRR', 'TIMELESS', 'TIMM10',
	'TK1', 'TMED5', 'TMEM106C', 'TMEM237', 'TMEM60', 'TMEM97', 'TMPO',
	'TMSB15A', 'TOP1', 'TOP2A', 'TPI1', 'TPR', 'TPRKB', 'TPX2',
	'TRA2B', 'TRAIP', 'TROAP', 'TTC28', 'TTF2', 'TTK', 'TUBA1B',
	'TUBA1C', 'TUBB', 'TUBB4B', 'TUBG1', 'TUBGCP3', 'TXNDC12', 'TYMS',
	'UBA2', 'UBB', 'UBE2C', 'UBE2D2', 'UBE2D3', 'UBE2I', 'UBE2N',
	'UBE2S', 'UBE2T', 'UHRF1', 'UNG', 'UQCC2', 'UQCC3', 'UQCRC1',
	'UQCRFS1', 'USP1', 'VBP1', 'VDAC3', 'VEZF1', 'VRK1', 'WAPL',
	'WDHD1', 'WDPCP', 'WDR34', 'WDR76', 'XPO1', 'XRCC4', 'XRCC5',
	'XRCC6', 'YAP1', 'YBX1', 'YEATS4', 'Z94721.1', 'ZFP36L1', 'ZGRF1',
	'ZMYM1', 'ZNF22', 'ZNF367', 'ZNF43', 'ZNF704', 'ZNF83', 'ZRANB3',
	'ZSCAN16-AS1', 'ZWINT'], dtype=object)

np.array([x[0] + x[1:].lower() for x in cc_genes_human], dtype=object)

array(['Abhd3', 'Ac016205.1', 'Ac073529.1', 'Ac084033.3', 'Ac087632.1',
       'Ac091057.6', 'Ac097534.2', 'Ac099850.2', 'Ac135586.2', 'Acaa2',
       'Acadm', 'Acp1', 'Actl6a', 'Acyp1', 'Adcy3', 'Add3', 'Adk', 'Ahcy',
       'Akirin2', 'Akr7a2', 'Al359513.1', 'Al449266.1', 'Al513165.2',
       'Anapc11', 'Anln', 'Anp32a', 'Anp32b', 'Anp32e', 'Ap001347.1',
       'Apold1', 'Arhgap11a', 'Arhgef39', 'Arid1a', 'Arl6ip1', 'Arl6ip6',
       'Armc1', 'Arpp19', 'Asf1b', 'Aspm', 'Asrgl1', 'Atad2', 'Atad5',
       'Atp1b3', 'Aurka', 'Aurkb', 'Banf1', 'Bard1', 'Baz1a', 'Baz1b',
       'Birc5', 'Blm', 'Bora', 'Brca1', 'Brca2', 'Brd8', 'Brip1', 'Btg3',
       'Bub1', 'Bub1b', 'Bub3', 'C11orf58', 'C19orf48', 'C1orf112',
       'C1orf35', 'C21orf58', 'C5orf34', 'Cacybp', 'Calm2', 'Camta1',
       'Carhsp1', 'Cbx1', 'Cbx3', 'Cbx5', 'Ccar1', 'Ccdc14', 'Ccdc167',
       'Ccdc18', 'Ccdc34', 'Ccdc77', 'Ccna1', 'Ccna2', 'Ccnb1', 'Ccnb2',
       'Ccne2', 'Ccnf', 'Cct4', 'Cct5', 'Cdc20', 'Cdc25b', 'Cdc25c',